In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import OneHotCategorical
from torch.distributions.kl import kl_divergence
import gymnasium as gym
from gymnasium.wrappers import RecordVideo
import numpy as np
import random
import wandb
from PIL import Image
import ale_py

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ============================================================================
# Utility Functions
# ============================================================================

def symlog(x):
    return torch.sign(x) * torch.log(torch.abs(x) + 1)

def lambda_return(rewards, values, continues, bootstrap, lambda_=0.95):
    """Compute lambda returns properly."""
    # rewards: [B, H], values: [B, H], continues: [B, H], bootstrap: [B]
    next_values = torch.cat([values[:, 1:], bootstrap[:, None]], dim=1)
    targets = rewards + continues * next_values
    
    outputs = []
    last = bootstrap
    for t in reversed(range(rewards.shape[1])):
        last = targets[:, t] + continues[:, t] * lambda_ * (last - next_values[:, t])
        outputs.append(last)
    
    returns = torch.stack(list(reversed(outputs)), dim=1)
    return returns

# ============================================================================
# Neural Network Components
# ============================================================================

class Conv2dSame(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, bias=True):
        super().__init__()
        self.stride = stride
        self.kernel_size = kernel_size
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride=stride, bias=bias)
    
    def forward(self, x):
        if self.stride == 1:
            pad = (self.kernel_size - 1) // 2
            x = F.pad(x, [pad, pad, pad, pad])
        return self.conv(x)

class ConvTranspose2dSame(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, bias=True):
        super().__init__()
        self.stride = stride
        self.conv = nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride=stride, bias=bias)
    
    def forward(self, x):
        out = self.conv(x)
        if self.stride == 2:
            pad = 1
            out = out[:, :, pad:-pad, pad:-pad]
        return out

class Encoder(nn.Module):
    def __init__(self, out_dim=1024):
        super().__init__()

        self.conv = nn.Sequential(
            nn.Conv2d(3, 32, 4, stride=2),   # 64 → 31
            nn.ReLU(),
            nn.Conv2d(32, 64, 4, stride=2),  # 31 → 14
            nn.ReLU(),
            nn.Conv2d(64, 128, 4, stride=2), # 14 → 6
            nn.ReLU(),
            nn.Conv2d(128, 128, 4, stride=2),# 6 → 2
            nn.ReLU()
        )

        self.fc = nn.Linear(128 * 2 * 2, out_dim)  

    def forward(self, obs):
        x = obs / 255.0
        x = self.conv(x)
        x = x.flatten(start_dim=1)
        x = self.fc(x)
        return x


class Decoder(nn.Module):
    def __init__(self, feat_dim):
        super().__init__()

        self.fc = nn.Linear(feat_dim, 1024)

        self.deconv = nn.Sequential(
            nn.ConvTranspose2d(1024, 128, 5, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, 5, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, 6, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 3, 6, stride=2),
        )

    def forward(self, feat):
        x = self.fc(feat)
        x = x.view(-1, 1024, 1, 1)
        x = self.deconv(x)
        return x

class RSSM(nn.Module):
    def __init__(self, action_dim, embed_dim, deter_size=1024, stoch_size=32, hidden_size=512, num_classes=32):
        super().__init__()
        self.deter_size = deter_size
        self.stoch_size = stoch_size
        self.num_classes = num_classes
        
        self.prior_net = nn.Sequential(
            nn.Linear(deter_size, hidden_size),
            nn.LayerNorm(hidden_size),
            nn.GELU(),
            nn.Linear(hidden_size, stoch_size * num_classes)
        )
        
        # Now using the passed embed_dim
        self.posterior_net = nn.Sequential(
            nn.Linear(deter_size + embed_dim, hidden_size),
            nn.LayerNorm(hidden_size),
            nn.GELU(),
            nn.Linear(hidden_size, stoch_size * num_classes)
        )
        
        self.gru = nn.GRUCell(stoch_size * num_classes + action_dim, deter_size)
    
    def initial_state(self, batch_size):
        return {
            'deter': torch.zeros(batch_size, self.deter_size).to(device),
            'stoch': torch.zeros(batch_size, self.stoch_size, self.num_classes).to(device)
        }
    
    def get_dist(self, logits):
        logits = logits.reshape(*logits.shape[:-1], self.stoch_size, self.num_classes)
        return OneHotCategorical(logits=logits)
    
    def observe(self, embed, action, state):
        stoch_flat = state['stoch'].reshape(state['stoch'].shape[0], -1)
        deter = self.gru(torch.cat([stoch_flat, action], dim=-1), state['deter'])
        
        post_logits = self.posterior_net(torch.cat([deter, embed], dim=-1))
        post_dist = self.get_dist(post_logits)
        stoch = post_dist.sample() + post_dist.probs - post_dist.probs.detach()
        
        prior_logits = self.prior_net(deter)
        
        return {'deter': deter, 'stoch': stoch}, prior_logits, post_logits
    
    def imagine(self, action, state):
        stoch_flat = state['stoch'].reshape(state['stoch'].shape[0], -1)
        deter = self.gru(torch.cat([stoch_flat, action], dim=-1), state['deter'])
        
        prior_logits = self.prior_net(deter)
        prior_dist = self.get_dist(prior_logits)
        stoch = prior_dist.sample() + prior_dist.probs - prior_dist.probs.detach()
        
        return {'deter': deter, 'stoch': stoch}
    
    def get_feat(self, state):
        stoch = state['stoch']
        deter = state['deter']
        
        # Handle both single step [B, D] and sequence [B, T, D] cases
        if stoch.dim() == 3:  # [B, stoch_size, num_classes]
            stoch_flat = stoch.reshape(stoch.shape[0], -1)
        elif stoch.dim() == 4:  # [B, T, stoch_size, num_classes]
            stoch_flat = stoch.reshape(*stoch.shape[:2], -1)
        else:
            raise ValueError(f"Unexpected stoch dimensions: {stoch.shape}")
        
        return torch.cat([deter, stoch_flat], dim=-1)

class Actor(nn.Module):
    def __init__(self, feat_size, action_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(feat_size, 512),
            nn.LayerNorm(512),
            nn.GELU(),
            nn.Linear(512, 512),
            nn.LayerNorm(512),
            nn.GELU(),
            nn.Linear(512, action_dim)
        )
    
    def forward(self, feat):
        return OneHotCategorical(logits=self.net(feat))

class Critic(nn.Module):
    def __init__(self, feat_size):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(feat_size, 512),
            nn.LayerNorm(512),
            nn.GELU(),
            nn.Linear(512, 512),
            nn.LayerNorm(512),
            nn.GELU(),
            nn.Linear(512, 1)
        )
    
    def forward(self, feat):
        return self.net(feat).squeeze(-1)

# ============================================================================
# DreamerV3 Agent
# ============================================================================

class DreamerV3Agent:
    def __init__(self, action_dim, obs_shape=(3, 64, 64), lr=3e-4):
        self.action_dim = action_dim
        
        # Create encoder to get actual embed dimension
        self.encoder = Encoder().to(device)
        
        # Get embed dimension by doing a forward pass
        with torch.no_grad():
            dummy_obs = torch.zeros(1, *obs_shape).to(device)
            embed_dim = self.encoder(dummy_obs).shape[-1]
        
        print(f"📐 Encoder output dimension: {embed_dim}")
        
        feat_size = 1024 + 32*32  # deter_size + stoch_size * num_classes
        
        self.rssm = RSSM(action_dim, embed_dim=embed_dim).to(device)
        self.decoder = Decoder(feat_size).to(device)
        self.reward_pred = Critic(feat_size).to(device)
        self.continue_pred = Critic(feat_size).to(device)
        self.actor = Actor(feat_size, action_dim).to(device)
        self.critic = Critic(feat_size).to(device)
        
        world_params = (list(self.encoder.parameters()) + 
                       list(self.rssm.parameters()) + 
                       list(self.decoder.parameters()) +
                       list(self.reward_pred.parameters()) +
                       list(self.continue_pred.parameters()))
        
        self.world_opt = optim.Adam(world_params, lr=lr)
        self.actor_opt = optim.Adam(self.actor.parameters(), lr=lr)
        self.critic_opt = optim.Adam(self.critic.parameters(), lr=lr)
    
    def act(self, obs, state, training=True):
        with torch.no_grad():
            embed = self.encoder(torch.FloatTensor(obs).unsqueeze(0).to(device))
            
            if state is None:
                state = self.rssm.initial_state(1)
            
            action_dummy = torch.zeros(1, self.action_dim).to(device)
            state, _, _ = self.rssm.observe(embed, action_dummy, state)
            
            feat = self.rssm.get_feat(state)
            action_dist = self.actor(feat)
            
            if training:
                action = action_dist.sample()
            else:
                action = F.one_hot(action_dist.probs.argmax(dim=-1), self.action_dim).float()
            
            return action.cpu().numpy()[0].argmax(), state
    
    def train_batch(self, obs, actions, rewards, dones):
        B, T = obs.shape[:2]
    
        # =====================================================
        # Encode observations
        # =====================================================
        embed = self.encoder(obs.reshape(B * T, *obs.shape[2:]))
        embed = embed.reshape(B, T, -1)
    
        # =====================================================
        # World Model Rollout
        # =====================================================
        state = self.rssm.initial_state(B)
        states, priors, posts = [], [], []
    
        for t in range(T):
            state, prior, post = self.rssm.observe(embed[:, t], actions[:, t], state)
            states.append(state)
            priors.append(prior)
            posts.append(post)
    
        deter = torch.stack([s["deter"] for s in states], dim=1)
        stoch = torch.stack([s["stoch"] for s in states], dim=1)
    
        feat = self.rssm.get_feat({"deter": deter, "stoch": stoch})
    
        # =====================================================
        # World Model Losses
        # =====================================================
        obs_flat = obs.reshape(B * T, *obs.shape[2:]) / 255.0
        recon = self.decoder(feat.reshape(B * T, -1))
        recon_loss = F.mse_loss(recon, obs_flat)
    
        feat_flat = feat.reshape(B * T, -1)
        pred_reward = self.reward_pred(feat_flat).reshape(B, T)
        reward_loss = F.mse_loss(pred_reward, rewards)
    
        pred_continue = self.continue_pred(feat_flat).reshape(B, T)
        continue_target = 1.0 - dones.float()
        continue_loss = F.binary_cross_entropy_with_logits(
            pred_continue, continue_target
        )
    
        prior_dist = self.rssm.get_dist(torch.stack(priors, 1))
        post_dist = self.rssm.get_dist(torch.stack(posts, 1))
        kl = kl_divergence(post_dist, prior_dist).mean()
        kl_loss = torch.maximum(kl, torch.tensor(1.0, device=device))
    
        world_loss = recon_loss + reward_loss + continue_loss + kl_loss
    
        self.world_opt.zero_grad()
        world_loss.backward()
        torch.nn.utils.clip_grad_norm_(
            list(self.encoder.parameters())
            + list(self.rssm.parameters())
            + list(self.decoder.parameters())
            + list(self.reward_pred.parameters())
            + list(self.continue_pred.parameters()),
            100.0,
        )
        self.world_opt.step()
    
        # =====================================================
        # Imagination (NO GRAD STOP HERE)
        # =====================================================
        with torch.no_grad():
            imag_state = {
                "deter": deter[:, -1],
                "stoch": stoch[:, -1],
            }
    
        imag_states = [imag_state]
        horizon = 15
    
        for _ in range(horizon):
            feat_i = self.rssm.get_feat(imag_state)
            action = self.actor(feat_i).sample()
            imag_state = self.rssm.imagine(action, imag_state)
            imag_states.append(imag_state)
    
        imag_deter = torch.stack([s["deter"] for s in imag_states], dim=1)
        imag_stoch = torch.stack([s["stoch"] for s in imag_states], dim=1)
        imag_feat = self.rssm.get_feat({"deter": imag_deter, "stoch": imag_stoch})
    
        # =====================================================
        # Actor Update (USES IMAGINATION GRAPH)
        # =====================================================
        imag_feat_flat = imag_feat.reshape(-1, imag_feat.shape[-1])
        imag_reward = self.reward_pred(imag_feat_flat).reshape(B, horizon + 1)
        imag_value = self.critic(imag_feat_flat).reshape(B, horizon + 1)
        imag_continue = torch.sigmoid(
            self.continue_pred(imag_feat_flat)
        ).reshape(B, horizon + 1)
    
        returns = lambda_return(
            imag_reward[:, :-1],
            imag_value[:, :-1],
            imag_continue[:, :-1],
            imag_value[:, -1],
            lambda_=0.95,
        )
    
        discount = torch.cumprod(
            torch.cat(
                [torch.ones(B, 1, device=device), imag_continue[:, :-1]], dim=1
            ),
            dim=1,
        )
    
        actor_loss = -(discount[:, :-1] * returns.detach()).mean()
    
        self.actor_opt.zero_grad()
        actor_loss.backward()
        torch.nn.utils.clip_grad_norm_(self.actor.parameters(), 100.0)
        self.actor_opt.step()
    
        # =====================================================
        # Critic Update (NEW GRAPH — DETACHED)
        # =====================================================
        imag_feat_detached = imag_feat.detach()
        imag_feat_flat_detached = imag_feat_detached.reshape(
            -1, imag_feat_detached.shape[-1]
        )
    
        imag_value_detached = self.critic(imag_feat_flat_detached).reshape(
            B, horizon + 1
        )
    
        critic_loss = F.mse_loss(
            imag_value_detached[:, :-1], returns.detach()
        )
    
        self.critic_opt.zero_grad()
        critic_loss.backward()
        torch.nn.utils.clip_grad_norm_(self.critic.parameters(), 100.0)
        self.critic_opt.step()
    
        return {
            "world_loss": world_loss.item(),
            "recon_loss": recon_loss.item(),
            "reward_loss": reward_loss.item(),
            "kl_loss": kl_loss.item(),
            "actor_loss": actor_loss.item(),
            "critic_loss": critic_loss.item(),
        }

# ============================================================================
# Replay Buffer
# ============================================================================

class ReplayBuffer:
    def __init__(self, capacity=50000, seq_len=50):
        self.buffer = []
        self.capacity = capacity
        self.seq_len = seq_len
    
    def add(self, obs, act, rew, done):
        self.buffer.append((obs, act, rew, done))
        if len(self.buffer) > self.capacity:
            self.buffer.pop(0)
    
    def sample(self, batch_size):
        if len(self.buffer) < self.seq_len + 1:
            return None
        
        obs_b, act_b, rew_b, done_b = [], [], [], []
        
        for _ in range(batch_size):
            idx = random.randint(0, len(self.buffer) - self.seq_len - 1)
            seq = self.buffer[idx:idx + self.seq_len]
            
            obs_b.append(np.stack([s[0] for s in seq]))
            act_b.append(np.array([s[1] for s in seq]))
            rew_b.append(np.array([s[2] for s in seq]))
            done_b.append(np.array([s[3] for s in seq]))
        
        return (
            torch.FloatTensor(np.stack(obs_b)).to(device),
            torch.FloatTensor(np.stack(act_b)).to(device),
            torch.FloatTensor(np.stack(rew_b)).to(device),
            torch.FloatTensor(np.stack(done_b)).to(device)
        )

def preprocess_obs(obs):
    img = Image.fromarray(obs).resize((64, 64), Image.BILINEAR)
    return np.transpose(np.array(img), (2, 0, 1))

# ============================================================================
# Load and Evaluate Model
# ============================================================================

def load_model(checkpoint_path, action_dim):
    """Load a saved model for evaluation or continued training."""
    agent = DreamerV3Agent(action_dim=action_dim)
    
    checkpoint = torch.load(checkpoint_path, map_location=device)
    agent.actor.load_state_dict(checkpoint['actor'])
    agent.critic.load_state_dict(checkpoint['critic'])
    agent.encoder.load_state_dict(checkpoint['encoder'])
    agent.rssm.load_state_dict(checkpoint['rssm'])
    
    if 'decoder' in checkpoint:
        agent.decoder.load_state_dict(checkpoint['decoder'])
    if 'reward_pred' in checkpoint:
        agent.reward_pred.load_state_dict(checkpoint['reward_pred'])
    if 'continue_pred' in checkpoint:
        agent.continue_pred.load_state_dict(checkpoint['continue_pred'])
    
    print(f"✅ Model loaded from {checkpoint_path}")
    if 'episode' in checkpoint:
        print(f"   Episode: {checkpoint['episode']}, Reward: {checkpoint.get('reward', 'N/A')}")
    
    return agent, checkpoint

def evaluate_model(checkpoint_path, num_episodes=10):
    """Evaluate a saved model."""
    # Try different environment names
    env_names = ['SpaceInvadersNoFrameskip-v4', 'SpaceInvaders-v4', 'SpaceInvaders-v0', 'ALE/SpaceInvaders-v5']
    env = None
    for env_name in env_names:
        try:
            env = gym.make(env_name, render_mode="rgb_array")
            print(f"✅ Using environment: {env_name}")
            break
        except:
            continue
    
    if env is None:
        raise RuntimeError("Could not create environment")
    
    agent, checkpoint = load_model(checkpoint_path, env.action_space.n)
    
    rewards = []
    for episode in range(num_episodes):
        obs, _ = env.reset()
        obs = preprocess_obs(obs)
        state = None
        ep_reward = 0
        done = False
        
        while not done:
            action_idx, state = agent.act(obs, state, training=False)
            next_obs, reward, term, trunc, _ = env.step(action_idx)
            done = term or trunc
            next_obs = preprocess_obs(next_obs)
            obs = next_obs
            ep_reward += reward
        
        rewards.append(ep_reward)
        print(f"Episode {episode + 1}/{num_episodes}: Reward = {ep_reward:.1f}")
    
    print(f"\nAverage Reward: {np.mean(rewards):.1f} ± {np.std(rewards):.1f}")
    env.close()
    return rewards

# ============================================================================
# Training Loop
# ============================================================================

def train(resume_from=None, max_episodes=300):
    """
    Train the DreamerV3 agent.
    
    Args:
        resume_from: Path to checkpoint to resume from (e.g., "checkpoints/recovery.pt")
        max_episodes: Maximum number of episodes to train (default: 300 for Kaggle)
    """
    # Initialize wandb
    # wandb.init(
    #     project="mahmoud-dreamer-v3-spaceinvaders",
    #     config={
    #         "lr": 3e-4,
    #         "horizon": 15,
    #         "batch_size": 16,
    #         "seq_len": 50,
    #         "buffer_size": 50000
    #     },
    #     resume="allow" if resume_from else False
    #     mode="disabled"
    # )
    
    env_name = 'SpaceInvadersNoFrameskip-v4'
    
    env = gym.make(env_name, render_mode="rgb_array")
    # env = gym.wrappers.AtariPreprocessing(
    #     env,
    #     frame_skip=1,          
    #     grayscale_obs=True,
    #     screen_size=64
    # )
    print(f"✅ Successfully created environment: {env_name}")
    print(f"   Action space: {env.action_space}")
    print(f"   Observation space: {env.observation_space}")

    
    if env is None:
        print(f"\n❌ Could not create any SpaceInvaders environment!")
        print(f"Last error: {last_error}")
        print("\n🔧 Please run this in Cell 1:")
        print("!pip install -q 'gymnasium==0.29.1' 'ale-py==0.8.1' 'shimmy[atari]'")
        print("import ale_py")
        print("ale_py.roms.utils.import_roms()")
        raise RuntimeError("Environment creation failed")
    
    try:
        env = RecordVideo(env, "./videos", episode_trigger=lambda x: x % 50 == 0)
    except Exception as e:
        print(f"⚠️ Video recording failed: {e}, continuing without recording...")
    
    # Get observation shape from environment
    obs_sample, _ = env.reset()
    obs_preprocessed = preprocess_obs(obs_sample)
    obs_shape = obs_preprocessed.shape
    print(f"📐 Preprocessed observation shape: {obs_shape}")
    
    agent = DreamerV3Agent(action_dim=env.action_space.n, obs_shape=obs_shape)
    buffer = ReplayBuffer()
    
    global_step = 0
    episode_rewards = []
    best_reward = -float('inf')
    start_episode = 0
    
    # Resume from checkpoint if specified
    if resume_from and os.path.exists(resume_from):
        print(f"📂 Resuming from {resume_from}")
        checkpoint = torch.load(resume_from, map_location=device)
        agent.actor.load_state_dict(checkpoint['actor'])
        agent.critic.load_state_dict(checkpoint['critic'])
        agent.encoder.load_state_dict(checkpoint['encoder'])
        agent.rssm.load_state_dict(checkpoint['rssm'])
        if 'decoder' in checkpoint:
            agent.decoder.load_state_dict(checkpoint['decoder'])
        if 'reward_pred' in checkpoint:
            agent.reward_pred.load_state_dict(checkpoint['reward_pred'])
        if 'continue_pred' in checkpoint:
            agent.continue_pred.load_state_dict(checkpoint['continue_pred'])
        start_episode = checkpoint.get('episode', 0) + 1
        episode_rewards = checkpoint.get('episode_rewards', [])
        best_reward = checkpoint.get('best_reward', -float('inf'))
        print(f"   Resuming from episode {start_episode}")
        print(f"   Best reward so far: {best_reward:.1f}")
    
    # Create checkpoints directory
    os.makedirs("checkpoints", exist_ok=True)
    
    # Time tracking
    import time
    start_time = time.time()
    
    for episode in range(start_episode, start_episode + max_episodes):
        obs, _ = env.reset()
        obs = preprocess_obs(obs)
        
        state = None
        ep_reward = 0
        done = False
        step = 0
        
        while not done:
            # Select action
            action_idx, state = agent.act(obs, state, training=True)
            act_onehot = np.zeros(env.action_space.n)
            act_onehot[action_idx] = 1.0
            
            # Step environment
            next_obs, reward, term, trunc, _ = env.step(action_idx)
            done = term or trunc
            next_obs = preprocess_obs(next_obs)
            
            # Store in buffer
            buffer.add(obs, act_onehot, reward, done)
            
            obs = next_obs
            ep_reward += reward
            step += 1
            global_step += 1
            
            # Train
            if step % 5 == 0 and len(buffer.buffer) > 1000:
                batch = buffer.sample(16)
                if batch:
                    metrics = agent.train_batch(*batch)
                    #wandb.log(metrics, step=global_step)
                    
                    # Log reconstruction occasionally
                    if global_step % 500 == 0:
                        with torch.no_grad():
                            feat = agent.rssm.get_feat(state)
                            recon = agent.decoder(feat)
                            # wandb.log({
                            #     "reconstruction": wandb.Image(
                            #         recon[0].cpu().numpy().transpose(1, 2, 0)
                            #     )
                            # }, step=global_step)
        
        episode_rewards.append(ep_reward)
        avg_reward = np.mean(episode_rewards[-10:]) if len(episode_rewards) >= 10 else ep_reward
        
        # Time estimation
        elapsed_time = time.time() - start_time
        episodes_done = episode - start_episode + 1
        time_per_episode = elapsed_time / episodes_done
        remaining_episodes = start_episode + max_episodes - episode - 1
        estimated_remaining = time_per_episode * remaining_episodes
        
        # wandb.log({
        #     "episode_reward": ep_reward,
        #     "avg_reward_10": avg_reward,
        #     "episode": episode,
        #     "time_per_episode": time_per_episode,
        # }, step=global_step)
        
        print(f"Episode {episode}: Reward = {ep_reward:.1f}, "
              f"Avg (last 10) = {avg_reward:.1f}, Steps = {global_step}")
        print(f"  Time: {time_per_episode:.1f}s/ep, "
              f"Remaining: {estimated_remaining/3600:.1f}h")
        
        # Save best model
        if ep_reward > best_reward:
            best_reward = ep_reward
            torch.save({
                'episode': episode,
                'reward': ep_reward,
                'actor': agent.actor.state_dict(),
                'critic': agent.critic.state_dict(),
                'encoder': agent.encoder.state_dict(),
                'rssm': agent.rssm.state_dict(),
                'decoder': agent.decoder.state_dict(),
                'reward_pred': agent.reward_pred.state_dict(),
                'continue_pred': agent.continue_pred.state_dict(),
            }, "checkpoints/best_model.pt")
            print(f"💾 New best model saved! Reward: {ep_reward:.1f}")
        
        # Save regular checkpoint every 50 episodes
        if episode % 50 == 0 and episode > 0:
            torch.save({
                'episode': episode,
                'reward': ep_reward,
                'actor': agent.actor.state_dict(),
                'critic': agent.critic.state_dict(),
                'encoder': agent.encoder.state_dict(),
                'rssm': agent.rssm.state_dict(),
                'decoder': agent.decoder.state_dict(),
                'reward_pred': agent.reward_pred.state_dict(),
                'continue_pred': agent.continue_pred.state_dict(),
                'episode_rewards': episode_rewards,
            }, f"checkpoints/checkpoint_ep{episode}.pt")
            print(f"💾 Checkpoint saved at episode {episode}")
        
        # Save recovery checkpoint every 10 episodes (overwrites)
        if episode % 10 == 0:
            torch.save({
                'episode': episode,
                'reward': ep_reward,
                'actor': agent.actor.state_dict(),
                'critic': agent.critic.state_dict(),
                'encoder': agent.encoder.state_dict(),
                'rssm': agent.rssm.state_dict(),
                'decoder': agent.decoder.state_dict(),
                'reward_pred': agent.reward_pred.state_dict(),
                'continue_pred': agent.continue_pred.state_dict(),
                'episode_rewards': episode_rewards,
                'best_reward': best_reward,
            }, "checkpoints/recovery.pt")
    
    # Save final model
    torch.save({
        'episode': episode,
        'reward': ep_reward,
        'actor': agent.actor.state_dict(),
        'critic': agent.critic.state_dict(),
        'encoder': agent.encoder.state_dict(),
        'rssm': agent.rssm.state_dict(),
        'decoder': agent.decoder.state_dict(),
        'reward_pred': agent.reward_pred.state_dict(),
        'continue_pred': agent.continue_pred.state_dict(),
        'episode_rewards': episode_rewards,
        'best_reward': best_reward,
    }, "checkpoints/final_model.pt")
    print(f"Final model saved!")
    
    env.close()
    #wandb.finish()

if __name__ == '__main__':
    
    evaluate_model("checkpoints/best_model.pt", num_episodes=10)

Using device: cuda
✅ Successfully created environment: SpaceInvadersNoFrameskip-v4
   Action space: Discrete(6)
   Observation space: Box(0, 255, (210, 160, 3), uint8)
📐 Preprocessed observation shape: (3, 64, 64)
📐 Encoder output dimension: 1024
Episode 0: Reward = 335.0, Avg (last 10) = 335.0, Steps = 2301
  Time: 63.3s/ep, Remaining: 17.6h
💾 New best model saved! Reward: 335.0
Episode 1: Reward = 180.0, Avg (last 10) = 180.0, Steps = 4618
  Time: 84.9s/ep, Remaining: 23.5h
Episode 2: Reward = 215.0, Avg (last 10) = 215.0, Steps = 7311
  Time: 97.3s/ep, Remaining: 27.0h
Episode 3: Reward = 385.0, Avg (last 10) = 385.0, Steps = 9864
  Time: 102.7s/ep, Remaining: 28.4h
💾 New best model saved! Reward: 385.0
Episode 4: Reward = 285.0, Avg (last 10) = 285.0, Steps = 13977
  Time: 120.1s/ep, Remaining: 33.2h
Episode 5: Reward = 50.0, Avg (last 10) = 50.0, Steps = 15478
  Time: 111.5s/ep, Remaining: 30.8h
Episode 6: Reward = 155.0, Avg (last 10) = 155.0, Steps = 17891
  Time: 111.6s/ep, Rem

KeyboardInterrupt: 